**Neural network**
Based on Solar_Forecasting.ipynb by
From GHI series (or other variable) makes prediction of next steps based on a rolling function called split_sequence that takes subseries of length N to predict the following M steps. Using a ANN aproach with 100 epochs.

We will use this implementation to forecast the following 24 months (from january 2021 to december 2022). The testing sample will be 2020 (for which the training data will be up to 2019).


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from time import time

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
print("Imports done")

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
#Data
VarNo=0
CoordNo=0
data_path="/home/bleon/Documents/DS4A/DS4A_Project/Processed_Data/"
Data=pd.read_csv(data_path+"MeteorCOLUnited.csv",usecols=["Year","Month","Day","Wind Speed","GHI","Temperature","Latitude","Longitude"])
MonthData=Data.groupby(["Year","Month","Latitude,Longitude"]).mean()[Data["Year"]<=2020]
MonthData["LatLon"]=list(zip(MonthData["Latitude"],MonthData["Longitude"]))

AllCoords=MonthData["LatLon"].unique().tolist()
Location=AllCoords[CoordNo]
Variables_to_plot=["Temperature","Wind Speed","GHI"]
Variable=Variables_to_plot[VarNo]

print("Data done")

In [ ]:
ForecastDF=MonthData[MonthData["LatLon"]==Location][["Year","Month",Variable]]
ForecastDF["datetime"]=pd.to_datetime(ForecastDF["Year"].astype("str")+"-"+ForecastDF["Month"].astype("str")+"-01")

In [ ]:
X = np.array(ForecastDF[Variable].tolist())
scaler = StandardScaler().fit(X.reshape(-1, 1))
X = scaler.transform(X.reshape(-1, 1))
X = X.ravel()
#
# X_test last year
# X_val Second to last year
# X_train all remaining year
n = len(X)
m=n-12
X_train = X[:m-24]
X_val = X[m-12:m]
X_test = X[m:]

print(X.shape)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
# seq=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
# print(split_sequence(seq,3,3))

In [ ]:
# Choose number of input and output steps
# Elegir número de pasos de entrada y salida
n_steps_in_d = 6     # 7 días previos
n_steps_out_d = 2      # predecir siguiente día

# Dividir en muestras
X_t, y_t = split_sequence(X_train, n_steps_in_d, n_steps_out_d)
X_v, y_v = split_sequence(X_val, n_steps_in_d, n_steps_out_d)

print(X_t.shape)
print(y_t.shape)
print(X_v.shape)
print(y_v.shape)

In [ ]:
n_features = 1
X_t = X_t.reshape((X_t.shape[0], X_t.shape[1], n_features))
y_t = y_t.reshape((y_t.shape[0], y_t.shape[1], n_features))
X_v = X_v.reshape((X_v.shape[0], X_v.shape[1], n_features))
y_v = y_v.reshape((y_v.shape[0], y_v.shape[1], n_features))

print(X_t.shape)
print(y_t.shape)
print(X_v.shape)
print(y_v.shape)

In [ ]:
# definir el modelo

# encoder
model_d = Sequential()
model_d.add(LSTM(12, input_shape=(n_steps_in_d, n_features)))
model_d.add(RepeatVector(n_steps_out_d))

# decoder
model_d.add(LSTM(12, return_sequences=True))
model_d.add(TimeDistributed(Dense(1)))
model_d.compile(optimizer='adam', loss='mse')

# entrenar the model
start = time()
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
history = model_d.fit(X_t, y_t, epochs=100, batch_size=128, verbose=1,
           validation_data=(X_v, y_v), callbacks=[early_stopping])
print("El modelo tardó %.2f segundos en entrenar."% (time() - start)) 

In [ ]:
model_d.summary()